In [1]:
import os
from openai import OpenAI

zai_client = OpenAI(
    api_key=os.getenv("ZAI_API_KEY"),
    base_url="https://api.z.ai/api/paas/v4/"
)

In [9]:
messages = [
    {"role": "user", "content": "What is 2+2?"}
]

response = zai_client.chat.completions.create(
    model='glm-4.7',
    messages=messages
)

print(response.choices[0].message.content)


4


In [4]:
story_prompt = [{"role": "user", "content": "Tell me a very short story (250 words)"}]

response = zai_client.chat.completions.create(
    model='glm-4.7',
    messages=story_prompt,
    stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='', flush=True)


The bell above the door chimed, shattering the heavy silence of the shop. Elias didn’t look up immediately. His eyes were magnified by thick lenses, focused intensely on the mainspring of a 19th-century pocket watch.

"Excuse me? Can you fix this?"

The voice was soft, young. Elias lowered his loupe. Standing before the counter was a girl, no older than twenty, clutching a silver wristwatch with a cracked face.

"It stopped when he passed," she whispered, placing it gently on the velvet mat. "My grandfather. It meant the world to him."

Elias picked it up. The metal was cold. He turned it over to inspect the case back. His breath hitched. The engraving was faint but legible: *To E, My heart, M.*

Elias hadn’t been called 'E' in forty years. He remembered giving this watch to Michael, his best friend, the night before Michael left for the war. He remembered Michael never returning, and the letter that arrived later saying Michael had met someone overseas.

"I can fix it," Elias said, hi

In [5]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]


In [8]:
messages = [
    {"role": "system", "content": "Extract the event information."},
    {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."}
]

response = zai_client.chat.completions.create(
    model='glm-4.7',
    messages=messages,
    response_format={
        "type": "json_schema",
        "json_schema": CalendarEvent.model_json_schema()
    }
)

print(response.choices[0].message.content)

Here is the event information extracted from the sentence:

*   **Event:** Science Fair
*   **Attendees:** Alice and Bob
*   **Date:** Friday


In [10]:
from anthropic import Anthropic

zai_anthropic_client = Anthropic(
    api_key=os.getenv("ZAI_API_KEY"),
    base_url="https://api.z.ai/api/anthropic"
)

In [11]:
messages = [{"role": "user", "content": "What is 2+2?"}]

response = zai_anthropic_client.messages.create(
    model="glm-4.7",
    max_tokens=100,
    messages=messages
)
print(response.content[0].text)


The answer is **4**.


In [12]:
response = zai_anthropic_client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=100,
    messages=messages
)
print(response.content[0].text)


The answer to 2 + 2 is 4. 

Here's the step-by-step breakdown:
1. Start with the number 2.
2. Add another 2 to it.
3. The total is 4.

Let me know if you have any other questions!


In [14]:
response.model

'glm-4.7'

In [15]:
structured_output_tool = {
    "name": "final_result",
    "description": "Tool for structured output",
    "input_schema": CalendarEvent.model_json_schema()
}

In [30]:
instructions = "Extract the event information."

messages = [
    {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."}
]

response = zai_anthropic_client.messages.create(
    model="glm-4.7",
    max_tokens=1024,
    system=instructions,
    messages=messages,
    tools=[structured_output_tool],
    tool_choice={"type": "tool", "name": structured_output_tool['name']}
)


In [31]:
response.content[0]

ToolUseBlock(id='call_79e7e4611cd9400da4556937', input={'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}, name='final_result', type='tool_use')

In [32]:
event = CalendarEvent.model_validate(response.content[0].input)
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [33]:
import rag
index = rag.initialize_index()


Indexed 385 chunks from 95 documents


In [34]:
class ZaiRAG(rag.RAG):

    def llm(self, user_prompt):
        structured_output_tool = {
            "name": "final_result",
            "description": "Tool for structured output",
            "input_schema": self.output_type.model_json_schema()
        }
        
        messages = [{"role": "user", "content": user_prompt}]
        
        response = self.llm_client.messages.create(
            model=self.model_name,
            max_tokens=1024,
            system=self.rag_instructions,
            messages=messages,
            tools=[structured_output_tool],
            tool_choice={"type": "tool", "name": structured_output_tool['name']}
        )
        
        output = self.output_type.model_validate(response.content[0].input)
        return output

In [35]:
zai_rag = ZaiRAG(
    index,
    zai_anthropic_client,
    model_name='glm-4.7'
)

response = zai_rag.rag('llm as a judge')
print(response.answer[:50] + '...')


# LLM as a Judge

LLM as a judge is an evaluation ...
